<a href="https://colab.research.google.com/github/RodrigoMousinho/hello-world/blob/master/RM_Q2_Case_Mottu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##1. Carregamento dos Dados
Primeiro, precisamos carregar a tabela de dados no Google Colab. A partir do arquivo fornecido para a resoluçao do Case, utilizaremos a biblioteca pandas para ler os dados e plotly para a visualização gráfica.

In [36]:
# Importando as bibliotecas
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go

# Lendo o arquivo CSV
df = pd.read_csv('Q2.csv')  # Substitua pelo nome correto do arquivo

# Mostrando os primeiros 5 registros
df.head()

,Data,A/V,Estado,Valor de Entrada,Caução
0,01/08/24,Aluguel,GO,NaN,700.0
1,01/08/24,Aluguel,PR,NaN,500.0
2,01/08/24,Venda,GO,2500.0,NaN
3,01/08/24,Aluguel,AM,NaN,500.0
4,01/08/24,Aluguel,BA,NaN,500.0


##2. Pré-processamento dos Dados
Antes de realizar a análise, vamos cuidar para que os dados estejam limpos e preparados.

In [37]:
# Verificar informações básicas do dataframe
print(df.info())

# Verificar se há valores ausentes
print(df.isnull().sum())

# Preencher valores ausentes, se necessário
# Por exemplo, preencher NaN em 'Valor de Entrada' e 'Caução' com 0
df['Valor de Entrada'] = df['Valor de Entrada'].fillna(0)
df['Caução'] = df['Caução'].fillna(0)

# Confirmar que não há mais valores ausentes
print(df.isnull().sum())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2004 entries, 0 to 2003
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Data              2004 non-null   object 
 1   A/V               2004 non-null   object 
 2   Estado            2004 non-null   object 
 3   Valor de Entrada  778 non-null    float64
 4   Caução            1226 non-null   float64
dtypes: float64(2), object(3)
memory usage: 78.4+ KB
None
Data                   0
A/V                    0
Estado                 0
Valor de Entrada    1226
Caução               778
dtype: int64
Data                0
A/V                 0
Estado              0
Valor de Entrada    0
Caução              0
dtype: int64


# Item a)– Qual filial teve um melhor desempenho no período analisado?
 ## Seções 3. 4. e 5.

##3. Análise de Desempenho por Filial
Vamos calcular o número total de transações (aluguel e venda) e o faturamento total por filial (Estado).

###3.1. Número de Transações por Filial

In [38]:
# Contar o número de transações por Estado e tipo (Aluguel/Venda)
transacoes_por_estado = df.groupby(['Estado', 'A/V']).size().unstack(fill_value=0)

# Adicionar uma coluna com o total de transações por Estado
transacoes_por_estado['Total Transações'] = transacoes_por_estado.sum(axis=1)

# Ordenar as filiais pelo número total de transações, do maior para o menor
transacoes_por_estado = transacoes_por_estado.sort_values(by='Total Transações', ascending=False)

print(transacoes_por_estado)


A/V     Aluguel  Venda  Total Transações
Estado                                  
SP          108    134               242
SC          118     76               194
MG           86    100               186
PE          127     52               179
BA          114     36               150
PR          102     28               130
AL          105     18               123
RS           22     68                90
ES           38     32                70
PA           26     35                61
AM           30     29                59
GO           34     22                56
CE           30     22                52
MT           24     17                41
RN           25      9                34
SE           24      9                33
MS           25      8                33
MA           19     13                32
DF           23      9                32
PB           23      9                32
PI           19     12                31
AP           22      6                28
RJ           19 

Esse gráfico mostra o número de aluguéis e vendas para cada filial (estado) em barras agrupadas.

In [39]:
# Gráfico de barras - Número de transações por estado
fig_transacoes = px.bar(transacoes_por_estado.reset_index(),
                        x='Estado',
                        y=['Aluguel', 'Venda'],
                        title="Número de Transações por Estado (Filial)",
                        labels={'value': 'Número de Transações', 'Estado': 'Estado'},
                        barmode='group')
fig_transacoes.show()


###3.2. Cálculo do Faturamento por Filial
Para calcular o faturamento, precisamos considerar os valores de entrada e caução.

**Venda:** Consideramos apenas o Valor de Entrada.

**Aluguel:** Consideramos a Caução.

In [40]:
# Calcular faturamento por transação
# Venda: Faturamento = Valor de Entrada
# Aluguel: Faturamento = Caução
df['Faturamento'] = df.apply(lambda row: row['Valor de Entrada'] if row['A/V'] == 'Venda' else row['Caução'], axis=1)

# Agrupar o faturamento por Estado
faturamento_por_estado = df.groupby('Estado')['Faturamento'].sum().sort_values(ascending=False)

print(faturamento_por_estado)


Estado
SP    360650.0
MG    273950.0
SC    247050.0
PE    194800.0
RS    158700.0
BA    155300.0
PR    132750.0
ES    101700.0
AL     99200.0
PA     96900.0
AM     86200.0
GO     70900.0
CE     70100.0
MT     54100.0
MA     44700.0
PI     38700.0
RN     35500.0
DF     34800.0
PB     33720.0
SE     33600.0
MS     33150.0
TO     31800.0
RJ     31800.0
AP     26100.0
AC     23300.0
RR     19400.0
RO     17750.0
Name: Faturamento, dtype: float64


In [41]:
# Gráfico de barras - Faturamento total por estado
fig_faturamento = px.bar(faturamento_por_estado.reset_index(),
                         x='Estado',
                         y='Faturamento',
                         title="Faturamento Total por Estado (Filial)",
                         labels={'Faturamento': 'Faturamento Total (R$)', 'Estado': 'Estado'},
                         color='Faturamento')
fig_faturamento.show()


###3.3. Combinação das Métricas

Para uma análise mais completa, podemos combinar o número de transações e o faturamento.

In [42]:
# Combinar número de transações e faturamento em um único dataframe
analise_completa = transacoes_por_estado.join(faturamento_por_estado)

# Renomear a coluna de faturamento para maior clareza
analise_completa = analise_completa.rename(columns={'Faturamento': 'Total Faturamento'})

print(analise_completa)


        Aluguel  Venda  Total Transações  Total Faturamento
Estado                                                     
SP          108    134               242           360650.0
SC          118     76               194           247050.0
MG           86    100               186           273950.0
PE          127     52               179           194800.0
BA          114     36               150           155300.0
PR          102     28               130           132750.0
AL          105     18               123            99200.0
RS           22     68                90           158700.0
ES           38     32                70           101700.0
PA           26     35                61            96900.0
AM           30     29                59            86200.0
GO           34     22                56            70900.0
CE           30     22                52            70100.0
MT           24     17                41            54100.0
RN           25      9                34

##4. Apresentação dos Resultados
Agora, vamos apresentar as filiais ordenadas pelo desempenho, considerando tanto o número de transações quanto o faturamento.

###Interpretação dos Resultados:

**Número de Transações:**  Filiais com maior número de transações indicam maior atividade, seja em vendas ou aluguéis.

**Faturamento Total:** Filiais com maior faturamento demonstram maior geração de receita.

**Análise Completa:** Combinar ambas as métricas fornece uma visão mais abrangente do desempenho de cada filial.

In [43]:
# Exibir as filiais ordenadas pelo total de transações
print("Filiais ordenadas pelo número total de transações:")
print(transacoes_por_estado[['Aluguel', 'Venda', 'Total Transações']])

print("\nFiliais ordenadas pelo faturamento total:")
print(faturamento_por_estado)

print("\nAnálise completa:")
print(analise_completa)


Filiais ordenadas pelo número total de transações:
A/V     Aluguel  Venda  Total Transações
Estado                                  
SP          108    134               242
SC          118     76               194
MG           86    100               186
PE          127     52               179
BA          114     36               150
PR          102     28               130
AL          105     18               123
RS           22     68                90
ES           38     32                70
PA           26     35                61
AM           30     29                59
GO           34     22                56
CE           30     22                52
MT           24     17                41
RN           25      9                34
SE           24      9                33
MS           25      8                33
MA           19     13                32
DF           23      9                32
PB           23      9                32
PI           19     12                31
AP    

In [44]:
import plotly.graph_objs as go

# Criar a figura
fig = go.Figure()

# Adicionar as barras do Total de Transações no eixo Y da esquerda
fig.add_trace(go.Bar(
    x=analise_completa.index,  # Estados (filiais)
    y=analise_completa['Total Transações'],  # Número de transações
    name='Total de Transações',  # Nome da legenda
    marker_color='blue',  # Cor das barras de transações
    yaxis='y1'  # Associar ao primeiro eixo Y
))

# Adicionar as barras do Faturamento Total no eixo Y da direita
fig.add_trace(go.Bar(
    x=analise_completa.index,  # Estados (filiais)
    y=analise_completa['Total Faturamento'],  # Faturamento total
    name='Faturamento Total (R$)',  # Nome da legenda
    marker_color='green',  # Cor das barras de faturamento
    yaxis='y2'  # Associar ao segundo eixo Y
))

# Atualizar o layout para incluir dois eixos Y
fig.update_layout(
    title="Análise Completa: Total de Transações e Faturamento por Estado",  # Título do gráfico
    xaxis_title="Estado (Filial)",  # Título do eixo X
    yaxis=dict(
        title="Total de Transações",  # Título do eixo Y da esquerda
        titlefont=dict(color='blue'),  # Cor do título correspondente às barras de transações
        tickfont=dict(color='blue')  # Cor dos valores correspondentes
    ),
    yaxis2=dict(
        title="Faturamento Total (R$)",  # Título do eixo Y da direita
        titlefont=dict(color='green'),  # Cor do título correspondente às barras de faturamento
        tickfont=dict(color='green'),  # Cor dos valores correspondentes
        overlaying='y',  # Sobrepõe o primeiro eixo Y
        side='right'  # Coloca o eixo à direita
    ),
    barmode='group',  # Agrupamento de barras
    template='plotly_white'  # Tema do gráfico
)

# Exibir o gráfico
fig.show()



###Explicação:

**Eixo Y Duplo:**
O Total de Transações é exibido no eixo y à esquerda (em azul).
O Faturamento Total é exibido no eixo y à direita (em verde).

**Agrupamento de Barras:** As barras são agrupadas lado a lado, mas agora com cada métrica tendo seu próprio eixo vertical, o que resolve o problema de escala.

**Visualização:** Com isso, você pode comparar as filiais, mesmo que os valores de transações e faturamento estejam em escalas diferentes.

##5. Conclusão  Item a):#

Com base na análise, a filial de **São Paulo (SP)** apresentou o melhor desempenho, tanto em termos de **número de transações** quanto de **faturamento total**. Isso indica uma alta demanda por ambos os serviços de aluguel e venda de motos nessa região.


In [45]:
# Seção de Conclusão com todos os gráficos apresentados

# Exibir o gráfico de número de transações por filial
fig_transacoes.show()

# Exibir o gráfico de faturamento total por filial
fig_faturamento.show()

# Exibir o gráfico de análise completa (transações e faturamento)
fig.show() # Changed fig_analise_completa to fig

#Item b) – Qual filial faturou mais no período apresentado na tabela?

## Seções 6. e 7.

## 6. Análise do Faturamento Total por Filial:


###**Entendimento do Enunciado:**

O enunciado menciona que o faturamento mensal de cada moto, seja por aluguel ou por venda, é o mesmo.
Portanto, o número de motos vendidas ou alugadas será o fator decisivo para calcular o faturamento.

###**Estratégia de Cálculo:**

**Para Aluguel:** Cada moto alugada gera o mesmo faturamento mensal que uma moto vendida.

**Para Venda:** O faturamento mensal já está embutido no número de motos vendidas.
Precisamos somar o número de motos vendidas e alugadas por filial e multiplicar pelo faturamento mensal médio por moto.

Já calculamos anteriormente o número de transações por filial. Agora, vamos aplicar o conceito de **faturamento equivalente** para determinar qual filial faturou mais.

Obs: O conceito de **faturamento equivalente** refere-se à ideia de tratar diferentes fontes de receita (neste caso, vendas e aluguéis) como geradoras do mesmo nível de faturamento mensal, independentemente de suas diferenças operacionais. Isso é feito para facilitar a comparação entre dois tipos de negócios distintos, mas que, ao final, possuem impactos semelhantes no resultado financeiro mensal.

In [46]:
# Vamos calcular o faturamento mensal por filial, assumindo que o faturamento mensal por moto alugada é o mesmo que o de uma moto vendida

# Número total de transações (vendas e aluguéis) por filial
analise_completa['Total Motos'] = analise_completa['Total Transações']  # Total de motos = total de transações

# Vamos supor um faturamento mensal fixo (pode ser ajustado conforme os dados específicos)
faturamento_mensal_por_moto = 1000  # Esse valor é apenas ilustrativo; ajuste conforme necessário

# Calcular o faturamento total por filial
analise_completa['Faturamento Total Estimado'] = analise_completa['Total Motos'] * faturamento_mensal_por_moto

# Ordenar as filiais pelo faturamento total
analise_completa = analise_completa.sort_values(by='Faturamento Total Estimado', ascending=False)

# Exibir a tabela com o faturamento estimado por filial
print(analise_completa[['Total Transações', 'Total Motos', 'Faturamento Total Estimado']])


        Total Transações  Total Motos  Faturamento Total Estimado
Estado                                                           
SP                   242          242                      242000
SC                   194          194                      194000
MG                   186          186                      186000
PE                   179          179                      179000
BA                   150          150                      150000
PR                   130          130                      130000
AL                   123          123                      123000
RS                    90           90                       90000
ES                    70           70                       70000
PA                    61           61                       61000
AM                    59           59                       59000
GO                    56           56                       56000
CE                    52           52                       52000
MT        

In [47]:
# Gerar gráfico de barras com faturamento total estimado por filial
fig_faturamento_total = px.bar(
    analise_completa.reset_index(),  # Usamos reset_index para organizar os dados
    x='Estado',  # Eixo X com as filiais (estados)
    y='Faturamento Total Estimado',  # Eixo Y com o faturamento total estimado
    title="Faturamento Total Estimado por Filial",  # Título do gráfico
    labels={'Faturamento Total Estimado': 'Faturamento Total (R$)', 'Estado': 'Filial (Estado)'},  # Rótulos dos eixos
    color='Faturamento Total Estimado',  # Colorir as barras pelo valor de faturamento
    color_continuous_scale=px.colors.sequential.Viridis  # Paleta de cores
)

# Exibir o gráfico
fig_faturamento_total.show()


##7.Conclusão Item b)



Após calcular o faturamento total estimado por filial, considerando o conceito de faturamento equivalente para vendas e aluguéis, podemos concluir que:

A filial de **São Paulo (SP)** apresentou o **maior faturamento total**. Isso se deve ao fato de ser uma das filiais com o maior número de transações, tanto de vendas quanto de aluguéis.



# Insight sobre os Itens a) e b)

## Seção 8

## 8.Diferenças entre os Faturamentos dos Itens a) e b):

###Item a) – Faturamento por Filial:

O faturamento foi calculado a partir das transações efetivas (aluguel ou venda), utilizando os valores de entrada e caução.
Nesse caso, o faturamento reflete as condições específicas de cada transação (se foi parcelada ou à vista, por exemplo). Reflete o faturamento **real** das transações, considerando as diferentes condições de cada operação.

###Item b) – Faturamento Total Estimado:

Aqui, utilizamos o conceito de faturamento equivalente. Cada moto, seja vendida ou alugada, foi tratada como geradora do mesmo valor mensal de faturamento. Isso simplifica a análise, mas ignora as diferenças reais nas condições de pagamento.
Esse método fornece um valor mais homogêneo e serve para padronizar a comparação entre filiais. Aplica um **valor fixo de faturamento mensal** por moto (independentemente de ser venda ou aluguel) para padronizar a análise.

In [48]:
# Comparar o faturamento calculado no Item a) com o faturamento estimado do Item b)

# Coluna 'Faturamento Total' já calculada no Item a)
# Coluna 'Faturamento Total Estimado' calculada no Item b)
comparacao_faturamento = analise_completa[['Total Faturamento', 'Faturamento Total Estimado']].copy()

# Calcular a diferença entre os dois faturamentos
comparacao_faturamento['Diferença'] = comparacao_faturamento['Faturamento Total Estimado'] - comparacao_faturamento['Total Faturamento']

# Exibir a tabela de comparação
print(comparacao_faturamento)


        Total Faturamento  Faturamento Total Estimado  Diferença
Estado                                                          
SP               360650.0                      242000  -118650.0
SC               247050.0                      194000   -53050.0
MG               273950.0                      186000   -87950.0
PE               194800.0                      179000   -15800.0
BA               155300.0                      150000    -5300.0
PR               132750.0                      130000    -2750.0
AL                99200.0                      123000    23800.0
RS               158700.0                       90000   -68700.0
ES               101700.0                       70000   -31700.0
PA                96900.0                       61000   -35900.0
AM                86200.0                       59000   -27200.0
GO                70900.0                       56000   -14900.0
CE                70100.0                       52000   -18100.0
MT                54100.0

In [49]:
import plotly.graph_objs as go

# Criar a figura
fig_comparacao = go.Figure()

# Adicionar as barras do Faturamento Total do Item a)
fig_comparacao.add_trace(go.Bar(
    x=comparacao_faturamento.index,
    y=comparacao_faturamento['Total Faturamento'],
    name='Faturamento Total (Item a)',
    marker_color='blue'
))

# Adicionar as barras do Faturamento Total Estimado do Item b)
fig_comparacao.add_trace(go.Bar(
    x=comparacao_faturamento.index,
    y=comparacao_faturamento['Faturamento Total Estimado'],
    name='Faturamento Total Estimado (Item b)',
    marker_color='green'
))

# Atualizar o layout
fig_comparacao.update_layout(
    title="Comparação: Faturamento Total (Item a) vs Faturamento Total Estimado (Item b)",
    xaxis_title="Estado (Filial)",
    yaxis_title="Faturamento (R$)",
    barmode='group'
)

# Exibir o gráfico
fig_comparacao.show()


# Item c) – Desenvolvimento da Solução de "Growth" da Mottu

## Seções 9. 10. 11. e 12.


Como responsável pela área de Growth da Mottu, o objetivo é otimizar os processos de venda e aluguel de motos e acompanhar métricas de desempenho. Para isso, a solução envolveu simular o processo no aplicativo e recriar a estrutura da tabela de transações, cuidando para que todas as informações relevantes fossem registradas de forma organizada.

###Justificativa das Respostas aos Itens I. e II.:

**I.Tabela Única para Vendas e Aluguel:** Optou-se por manter uma tabela única, centralizando os dados de venda e aluguel para facilitar a comparação e o controle das transações. A estrutura foi ajustada para incluir colunas específicas para cada tipo de transação, sem perder a nitidez dos dados.

**II.Mapeamento do Funil de Vendas:** Foi mapeado um funil de vendas unificado, que reflete as etapas críticas da jornada do cliente no aplicativo, desde a escolha do plano até o agendamento da retirada. O funil permite identificar gargalos e melhorar a eficiência do processo.

##9. Simulação do Processo no Aplicativo:
A primeira etapa consistiu na utilização do aplicativo da Mottu para simular o processo de aluguel de uma moto. Essa simulação foi essencial para identificar as etapas específicas pelas quais o cliente passa, como a inserção da localização, o cadastro de informações pessoais, a verificação da CNH, e a escolha da moto e do plano.

##10. Mapeamento do Funil de Vendas:
Com base nas informações coletadas na simulação, foi possível mapear o funil de vendas, detalhando cada etapa do processo, desde a captação inicial do cliente até a conclusão da transação com o pagamento e agendamento da retirada da moto. O funil foi estruturado para refletir a jornada real do cliente e identificar os pontos críticos de decisão e ação.

**1. Inserção da Localização (Captação)**

A primeira coisa que o app solicita é a localização do usuário.
Para novos usuários, o app pergunta se ele já é cliente ou se é a primeira vez.

**2. Cadastro de Informações Pessoais (Engajamento)**

Usuários novos são solicitados a fornecer:
CPF
Nome completo
Data de nascimento
DDD e telefone
Leitura e aceitação dos termos de uso e contrato de locação.

**3. Verificação de Telefone e Endereço (Verificação)**

Após a verificação do número de celular, o app solicita:
Endereço completo: Rua, número, complemento, bairro, CEP, estado, cidade.

**4. Validação da CNH (Verificação)**

O usuário deve confirmar que possui uma CNH válida para motos e que está ativa no Brasil.

**5. Seleção do Local de Retirada (Engajamento)**

Após a verificação da CNH, o app redireciona o cliente para a página de seleção do local de retirada, onde ele escolhe o local mais conveniente.

**6. Escolha da Moto e Plano (Decisão)**

O cliente visualiza os modelos de motos disponíveis na região e escolhe a que melhor atende às suas necessidades.
A seguir, o cliente seleciona o plano mais adequado (por exemplo, plano "Conquiste Sport 0km").

**7. Resumo e Pagamento (Ação)**

O cliente revisa as escolhas na página "Suas Escolhas", onde vê o modelo da moto, o endereço de retirada e o valor a ser pago.
Ele escolhe o método de pagamento (Pix ou cartão de crédito).

**8. Agendamento e Retirada (Retenção)**

Após o pagamento, o cliente agenda a retirada da moto e recebe uma confirmação da data, hora e local.

##11. Recriação da Estrutura da Tabela:
A estrutura da tabela foi adaptada para capturar dados importantes em cada etapa do funil de vendas. Essa tabela foi projetada para acompanhar o progresso de cada cliente, registrando informações essenciais como CPF, endereço completo, confirmação da CNH válida, modelo da moto, plano escolhido, método de pagamento, e o estado da transação. Além disso, foram incluídas colunas para monitorar o uso de códigos de desconto e o agendamento da retirada.


In [50]:
import plotly.graph_objects as go

# Dados do funil de vendas com colunas adicionais
dados_funil_atualizado = {



    'Data da Transação': ['2024-10-01', '2024-10-02', '2024-10-03'],
    'CPF e Nome Completo': ['123.456.789-00 - João Silva', '987.654.321-00 - Maria Souza', '555.666.777-00 - Pedro Almeida'],
    'A/V (Aluguel/Venda)': ['Venda', 'Aluguel', 'Aluguel'],
    'Estado (Brasil)': ['SP', 'SP', 'SP'],  # Estado do Brasil
    'Endereço Completo': ['Rua A, 123, Bairro X, Cidade Y, SP, 12345-678', 'Rua B, 456, Bairro Z, Cidade W, RJ, 87654-321', 'Rua C, 789, Bairro V, Cidade Q, MG, 65432-987'],
    'CNH Válida': ['Sim', 'Sim', 'Sim'],
    'Modelo da Moto': ['Pop 110i', 'Mottu Sport', 'Mottu-E Max'],
    'Local de Retirada': ['Mottu Butantã', 'Mottu Limao - Zona Norte', 'Mottu Interlagos'],
    'Plano Selecionado': ['Conquiste Sport 0km', 'Ilimitado', 'Urbano Flex'],
    'Valor de Entrada (R$)': [2000, 0, 0],  # Aluguel não tem valor de entrada
    'Caução (R$)': [0, 1200, 800],  # Informações de caução para aluguel
    'Método de Pagamento': ['Pix', 'Pix', 'Cartão de Crédito'],
    'Estado da Transação': ['Concluído', 'Aguardando Pagamento', 'Concluído'],
    'Data de Retirada Agendada': ['2024-10-05', '2024-10-07', '2024-10-09'],
    'Código de Desconto Aplicado': ['Nenhum', 'CUPOM20', 'Nenhum']
}

# Criar a tabela com Plotly
fig = go.Figure(data=[go.Table(
    header=dict(values=['<b>Coluna</b>', '<b>Descrição</b>'],
                fill_color='paleturquoise',
                align='center',
                font=dict(color='black', size=12)),
    cells=dict(values=[list(dados_funil_atualizado.keys()), list(dados_funil_atualizado.values())],
               fill_color='lavender',
               align='left'))
])

# Atualizar o layout para melhorar a visualização
fig.update_layout(
    title="Tabela de Transações com Colunas Adicionais",
    width=1000,
    height=600
)

# Exibir a tabela
fig.show()


##12. Visualização do Funil de Vendas:

Para uma melhor compreensão das etapas e do fluxo de clientes ao longo do funil, foi criada uma visualização gráfica utilizando ferramentas de plotagem de dados. O gráfico do funil de vendas ilustra a queda no número de clientes à medida que avançam pelas etapas, fornecendo insights valiosos sobre possíveis gargalos ou pontos de melhoria no processo de vendas.

In [51]:
import plotly.graph_objs as go

# Atualizar os dados para refletir as etapas descritas do funil de vendas atualizadas
etapas_funil_atualizado = [
    'Inserção da Localização (Captação)',
    'Cadastro de Informações Pessoais (Engajamento)',
    'Verificação de Telefone e Endereço (Verificação)',
    'Validação da CNH (Verificação)',
    'Seleção do Local de Retirada (Engajamento)',
    'Escolha da Moto e Plano (Decisão)',
    'Resumo e Pagamento (Ação)',
    'Agendamento e Retirada (Retenção)'
]

# Atualizando os números de clientes com base nas informações fornecidas
clientes_por_etapa_atualizado = [1000, 950, 920, 900, 870, 830, 800, 750]  # Dados ajustados para as etapas

# Criar gráfico do funil de vendas atualizado com os novos dados
fig_funil_atualizado = go.Figure(go.Funnel(
    y=etapas_funil_atualizado,
    x=clientes_por_etapa_atualizado,
    textinfo="value+percent initial"
))

# Adicionar título e subtítulo
fig_funil_atualizado.update_layout(
    title={
        'text': "10. Mapeamento do Funil de Vendas",
        'x': 0.5,  # Centralizar o título
        'xanchor': 'center',
        'yanchor': 'top'
    },
    annotations=[{
        'text': "Obs: Os números de clientes por etapa são virtuais e estão a serviço da simulação",
        'xref': 'paper',
        'yref': 'paper',
        'showarrow': False,
        'x': 0.5,  # Centralizar o subtítulo
        'y': -0.2,
        'font': {'size': 12}
    }]
)

# Exibir o gráfico do funil atualizado
fig_funil_atualizado.show()

# Item d) – Proposta de Análise Adicional

## Seções 13. 14. e 15.

## 13. Análise da Retenção de Clientes (Aluguel vs. Venda)

Para desenvolver o problema, podemos destacar um ponto relevante que ainda não foi abordado nas questões anteriores: a **retenção de clientes** comparando os **perfis de aluguel e venda** de motos.

**Contexto:**

Atualmente, a Mottu é amplamente reconhecida pelo seu serviço de aluguel de motos, mas também está investindo em um novo modelo de venda de motos. Cada uma dessas transações apresenta dinâmicas diferentes que podem impactar diretamente na retenção de clientes. Isso ocorre devido às diferenças nos modelos de pagamento e nos requisitos para retirada dos veículos, tanto no aluguel quanto na venda.

**Venda de Motos:** O cliente pode retirar a moto apenas após o pagamento **integral** da entrada (seja à vista ou parcelado), o que pode criar um **gap temporal** até que a transação seja concluída, especialmente se o cliente optar pelo parcelamento da entrada em até 5 vezes. Após a retirada, o financiamento segue com parcelas mensais durante três anos.

**Aluguel de Motos:** Já no caso do aluguel, o pagamento é **semanal** e pré-pago, com a caução podendo ser parcelada ao longo das semanas de aluguel. A moto pode ser retirada logo após o pagamento da primeira parcela de aluguel e caução, criando uma **relação contínua** entre a Mottu e o cliente ao longo de um ano de contrato.

**Justificativa da Análise:**

A análise da **retenção de clientes** pode fornecer insights valiosos sobre **qual modelo de transação mantém o cliente mais engajado com a Mottu ao longo do tempo**.

**Retenção no Aluguel:**

O **modelo pré-pago semanal** do aluguel garante um contato contínuo e previsível entre o cliente e a Mottu, já que o cliente precisa fazer pagamentos semanais para continuar com o uso da moto. Isso pode **aumentar a retenção**, pois o cliente já está comprometido com um contrato de um ano, e a caução serve como uma garantia para a Mottu em caso de desistência.
Além disso, a **flexibilidade no parcelamento da caução** pode facilitar o acesso ao serviço e **reduzir barreiras** para novos clientes, o que pode impactar positivamente a retenção.

**Retenção na Venda:**

No modelo de venda, a relação com o cliente após a compra tende a ser menos frequente, já que o contato contínuo é limitado às **parcelas mensais** do financiamento. No entanto, a retenção aqui pode ser influenciada por um **bom serviço pós-venda** e a possibilidade de ofertas de **manutenção ou upgrades**.
O **período de espera** para a retirada do veículo, especialmente se o cliente opta por parcelar a entrada, pode ser um **ponto de desistência** ou um fator que afeta a retenção inicial.

**Perguntas de Análise:**

*   **O aluguel gera uma retenção maior devido à natureza contínua e pré-paga do contrato?**

*  **A venda de motos poderia beneficiar-se de um maior investimento em pós-venda e programas de fidelidade?**


**Estrutura da Tabela com Base na Retenção:**

Para melhor capturar essa análise, a tabela poderia incluir uma coluna que **marca se o cliente é um "cliente retornado"**, ou seja, se ele já realizou transações anteriores com a Mottu, para que possamos **medir taxas de retorno no aluguel e na venda**.

###Conclusão:
Ao comparar a **retenção de clientes** entre o modelo de aluguel e o de venda, podemos identificar qual deles gera maior **fidelidade** e buscar estratégias para melhorar o desempenho em ambos os casos. Por exemplo, **incentivos de fidelidade** podem ser oferecidos tanto para os clientes que alugam quanto para os que compram, visando prolongar a relação deles com a Mottu.

Essa análise não só ajudará a entender melhor o comportamento dos clientes, mas também a ajustar as estratégias de **pós-venda** e de **retenção**.


## 14. Análise de Preferências de Uso (Região Restrita vs. Circulação Livre)

Outro ponto relevante que quero abordar é a **diferença nas preferências de uso** entre os clientes que optam pelo aluguel de motos e aqueles que escolhem o plano de venda. Uma distinção importante entre esses dois produtos está na **liberdade de circulação:** os clientes de aluguel só podem circular na região do contrato, enquanto os clientes do plano de venda têm liberdade de trânsito em todo o território nacional.

**Justificativa da Análise:**

Essa diferença no uso regional das motos pode ter um impacto significativo nas decisões dos clientes ao escolher entre os planos de aluguel ou venda, e também pode influenciar a retenção e o comportamento de recompra. Para a Mottu, entender essas preferências pode ser crucial para ajustar a comunicação e posicionar melhor o produto de venda no mercado, especialmente para clientes que precisam de mais flexibilidade geográfica.

**Questões a Considerar:**

*   **Os clientes de aluguel estão satisfeitos com a restrição de circulação regional, ou muitos deles poderiam migrar para o plano de venda pela vantagem da liberdade de trânsito?**

*   **Como os benefícios do aluguel, como manutenção inclusa e seguro, impactam a decisão de permanência no plano de aluguel?**

*   **O plano de venda “Conquiste Minha Sport” está sendo percebido como uma alternativa atraente para aqueles que precisam de maior mobilidade?**

**Como essa análise pode ser feita:**

A tabela pode incluir uma **coluna adicional** para capturar o **tipo de uso da moto**, seja ele **restrito à região do contrato** (aluguel) ou **circulação livre pelo território nacional** (venda). Isso permitiria uma análise mais detalhada das preferências dos clientes e ajudaria a entender se há uma migração de perfil entre os usuários de aluguel e venda, e como a comunicação da Mottu pode ser ajustada para atrair novos compradores.

### Conclusão:

Ao adicionar uma análise das **preferências de uso**, podemos entender melhor o impacto da **restrição regional** no aluguel e a **liberdade de circulação** no plano de venda. Isso pode fornecer insights valiosos para ajustar as estratégias de marketing e para melhorar a divulgação do plano “Conquiste Minha Sport”, que oferece maior flexibilidade geográfica.

Esse ponto de análise pode destacar como a **circulação livre** no plano de venda pode ser um diferencial importante para atrair clientes que precisam de maior mobilidade, e como o **aluguel com benefícios adicionais** ainda é uma escolha forte para entregadores que operam em áreas restritas.

## 15.Conclusão Item d)

Ao integrar essas duas análises, podemos perceber que:

*   A **retenção de clientes** no aluguel pode ser maximizada ao manter a **flexibilidade no pagamento da caução** e o contato constante por meio dos pagamentos semanais, enquanto os benefícios do plano (como manutenção e seguro) asseguram que os clientes tenham uma experiência fluida e sem grandes preocupações operacionais.

*   Por outro lado, o **plano de venda** pode ser posicionado como uma opção mais vantajosa para clientes que buscam **liberdade de circulação** e **independência regional**, algo que pode ser explorado em campanhas de marketing voltadas para clientes que precisam de mobilidade nacional. O foco no financiamento sem análise de crédito também pode atrair um novo público, ampliando o alcance do produto.



*   **Estrutura da Tabela Integrada:**




In [2]:
# Dados da tabela com retenção do cliente e tipo de circulação
dados_tabela = {
    'Data': ['01/10/2024', '02/10/2024', '03/10/2024'],
    'A/V (Aluguel/Venda)': ['Venda', 'Aluguel', 'Aluguel'],
    'Estado (Brasil)': ['SP', 'RJ', 'MG'],
    'Cliente Retornado?': ['Sim', 'Não', 'Sim'],
    'Modelo da Moto': ['Mottu Sport', 'Mottu Sport', 'Mottu-E Max'],
    'Plano Selecionado': ['Conquiste Minha Sport', 'Ilimitado', 'Urbano Flex'],
    'Tipo de Circulação': ['Livre (Todo o Brasil)', 'Região do Contrato', 'Região do Contrato'],
    'Método de Pagamento': ['Cartão de Crédito', 'Pix', 'Cartão de Crédito'],
    'Valor de Entrada (R$)': ['2.000,00 (à vista)', '-', '-'],
    'Caução (R$)': ['-', '700,00 (Parcelado)', '500,00 (À vista)']
}

# Criar a tabela com Plotly, selecionando colunas principais
fig = go.Figure(data=[go.Table(
    header=dict(values=[
        '<b>Data</b>', '<b>A/V (Aluguel/Venda)</b>', '<b>Estado (Brasil)</b>',
        '<b>Cliente Retornado?</b>', '<b>Modelo da Moto</b>', '<b>Plano Selecionado</b>',
        '<b>Tipo de Circulação</b>', '<b>Método de Pagamento</b>', '<b>Valor de Entrada (R$)</b>', '<b>Caução (R$)</b>'
    ],
    fill_color='paleturquoise',
    align='center',
    font=dict(color='black', size=12)),
    cells=dict(values=[
        dados_tabela['Data'], dados_tabela['A/V (Aluguel/Venda)'], dados_tabela['Estado (Brasil)'],
        dados_tabela['Cliente Retornado?'], dados_tabela['Modelo da Moto'], dados_tabela['Plano Selecionado'],
        dados_tabela['Tipo de Circulação'], dados_tabela['Método de Pagamento'], dados_tabela['Valor de Entrada (R$)'], dados_tabela['Caução (R$)']
    ],
    fill_color='lavender',
    align='left'))
])

# Atualizar o layout para melhorar a visualização
fig.update_layout(
    title="Colunas Adicionais: Retenção do Cliente e Tipo de Circulação",
    width=1000,
    height=400
)

# Exibir a tabela
fig.show()